# Are England historically Root(ed) without Joe?

### A historical comparison of the dependence of the current England Cricket Team on Joe Root scoring runs

At present (September 2021) Joe Root is in the form of his life. The remainder of the England batting line-up, however, is not. The conjecture is that this England Test side is more dependent on Joe Root scoring big runs than nearly any other side in modern cricketing history. A suggested comparison is the reliance of the late 1990's / early 2000's West Indies side on Brian Lara. But is this conjecture borne out in the data? This notebook analyses year-by-year Test batting statistics (appreciating this may not be the most appropriate temporal window available) to determine whether England really are Root(ed) without Joe.

# Assumptions & Input Data

### Governing Assumptions

* The assumptions detailed are relevant across:-
    * All time periods - no temporal variation is acocunted for
    * All teams - no particular adjustment is made for changes in team situation (i.e. new Test playing nations like Afghanistan are treated the same as England and Australia)
    * All locations - no delineation is made between home and away matches
    * All players - no delineation is made between those players deemed batters, and those not
* This analysis considers only Men's Test Cricket currently. A similar statistical analysis for Women's Test Cricket is in the works


### Input data
* Input data (drawn from CricInfor StatsGuru platform) is assumed to be correct in full, and complete to the latest matches
* Total runs scored by each Test playing nation is extracted on a year-by-year basis
* Total runs scored by the top 50 run scorers across all test playing nations is extracted on a year-by-year basis. The top 50 has been selected as good representation of the likely top scorers for all nations, thereby capturing data necessary to analyse team reliance
* The temporal windowing on year-by-year data (i.e. assessment of calendar years) may not be the most appropriate window available. However, in the first instance, it is the most readily accesible window given data is categorised into calendar years.


# Methodology

The methodology to analyse the historical context of the England Team's current reliance on Joe Root's batting output is defined below. This question can be posed in a number of ways, such as:-

* Does Joe Root score a historically high proportion of England's test runs for the current team?
* What is the probability of England posting a large innings score, given Joe Root makes above 50? And how does this compare to historical probabilities for other player / team combinations?
* What is the probability of England posting a large innings score, given Joe Roo makes below 50? And how does this compare to historical probabilities for other player / team combinations?
* What is the average England innings total, given a score by Joe Root within defined intervals (i.e. 0-10, 10-20, 20-30 etc.)?And how does this compare to historical averages for other player / team combinations?

All questions posed above will tell us something above England's reliance on Root. However, they will also all provide subtly different information. For example, assessment of innings data will remove skew due to very-large scores. As noted when defining input data, a year-by-year cumulative approach is adopted, which neglects analyses on a inidivudal innings-by-innings basis. It is assumed that year-by-year statistics are a good approximation of match-by-match performance, however this bears further scrutiny in future analyses. Given the year-by-year approach, this analysis will initially focus on the proportion of runs scored in Tests by Joe Root (and others), relative to their team scores. 

Therefore, reliance here is defined as the proportion of player to team runs. The higher this proportion, the higher the team relies on said player. A naive assumptions may be that this proportion should be equal to 1/11 = 9.09 %. This of course does not account for the skills disparity between batters and non-batters, and therefore a more reasonable assumption (where we define numbers 1-7 as batters) may be ~12.1 %. This number derives from an assumption the top 7 will score 85 % of the teams runs, spread evenly across all players. Of course, we may refine this assumption further. However, for the purposes of this analysise, let us assume that a reasonably expected baseline for player/team reliance is ~12.1 %. Any figure higher than this would indicate a batter on whom the team may rely more than your average batter. To the extreme, we may consider "A Bannerman" as the limit of this reliance percentage. That is, the 67.35 % of Australian first innings runs that Charles Bannerman scored with his 165 runs out of 245 total in the first ever test match (also the first ever Test innings, defined many years post). If we consider player / team reliance for batters to lie somewhere on the range 12.1 % - 67.35 % we have a reasonable (albeit vague) guide of what to expect.

When we consider year-by-year cumulative figures we also tend to smooth out anomalies. For example, Kyle Mayers may have had a high reliance ranking after his double century in his first test match against Bangladesh in 2021. This has surely reduced in subsquent outings. The calendar year delineation is an arbitrary temporal window, but one that is regularly used in cricketing statistical analysis. We will stick with this windowing. 

The result of this analysis is a ranking of reliance across Men's Test match history. This means players may well appear in the ranking multiple times for multiple years. Back to our original question, we will use this ranking to determine if the current England Team does indeed rely in historically unprecedented manner on Joe Root. 

# Analysis

In [ ]:
# Root_analysis.py
# ###############################################
# Author: J Jorgensen
# Date: 06/09/2021
# Function: batsman4s
# This function analyses the historical batting player / team reliance for test cricket 
#################################################

In [66]:
# Import required modules
import numpy as np
import pandas as pd
import scipy
from matplotlib import pyplot as plt

In [44]:
# Set-up parameters for data ingest from CricInfo Statsguru
# Initialize url to ""
url = ""
suburl1 = "http://stats.espncricinfo.com/ci/engine/stats/index.html"

# Match type - set to tests
match_type = "?class=1;"

# Test teams
teams = ["Afghanistan", "Australia", "Bangladesh", "England", "ICC World XI", "India",
        "Ireland", "New Zealand", "Pakistan", "South Africa", "Sri Lanka", "West Indies",
        "Zimbabwe"]

# Setting calendar year span
spanmax = "spanmax1=31+Dec+"
spanmin = ";spanmin1=01+Jan+"
spanval = ";spanval1=span;"

# Setting results type to template, and stats type to batting
template = ";template=results;"
player_batting = "type=batting"
team_batting = "type=team;"
match = "view=match"

# Set year runs
years = np.arange(1877, 2022,1)
years = np.char.mod('%d', years)


In [137]:
# Ingest data from CricInfo Statsguru & Calculate calendar year reliance

# Set-up reliance database

reliance = np.zeros((len(years), len(teams)))
players = np.empty([len(years), len(teams)], dtype="S30")


for j in range(0,9):#len(teams)):
    
    for i in range(0,len(years)):

        # Set-up urls
        url_players = suburl1 + match_type + spanmax + years[i] + spanmin + years[i] + spanval + "team=" + str(j+1) + template + player_batting
        url_team = suburl1 + match_type + spanmax + years[i] + spanmin + years[i] + spanval + "team=" + str(j+1) + template + team_batting + match
        
        # Read data from url
        dfList_players = pd.read_html(url_players)
        dfList_team = pd.read_html(url_team)
        
        # Calculate maximum reliance for that team for that year, and harvest player name
        if dfList_team[2].iloc[0,0] == "No records available to match this query":
            reliance[i,j] = 0
            players[i,j] = ''
        else:
            df_players = dfList_players[2]
            df_team  = dfList_team[2]
            
            # Calculate maximum reliance for that team for that year, and harvest player name
            reliance[i,j] = df_players["Runs"][0] / sum(df_team["Runs"])
            players[i,j] = df_players["Player"][0]


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [145]:
df_players["Runs"]

0     413
1     303
2     278
3     209
4     207
5     187
6     181
7     164
8      88
9      65
10     58
11     55
12     52
13     51
14     44
15     43
16     42
17     41
18     31
19     31
20     29
21     28
22     24
23     23
24     23
25     20
26     18
27     17
28      9
29      8
30      7
31      7
32      6
33      4
34      -
Name: Runs, dtype: object